In [ ]:
#|default_exp arena_agent
"""
Arena-Buddy web chat + LLM agent (nbdev-ready)
============================================
Live chat UI (FastHTML + HTMX + Tailwind/daisyUI) that streams responses from an “Arena Buddy” LLM agent.

**Changelog vs. original**

* FastHTML attribute bug fixed (`hx-swap-oob`).
* Assistant HTML no longer double-escaped.
* Proper SSE via `EventSourceResponse` (sse-starlette).
* Race-safety (`asyncio.Lock`).
* Backslash-in-f-string SyntaxError removed.
* Structured into **small cells** for easier debugging in a notebook.
"""


'\nArena-Buddy web chat + LLM agent (nbdev-ready)\n============================================\nLive chat UI (FastHTML + HTMX + Tailwind/daisyUI) that streams responses from an “Arena Buddy” LLM agent.\n\n**Changelog vs. original**\n\n* FastHTML attribute bug fixed (`hx-swap-oob`).\n* Assistant HTML no longer double-escaped.\n* Proper SSE via `EventSourceResponse` (sse-starlette).\n* Race-safety (`asyncio.Lock`).\n* Backslash-in-f-string SyntaxError removed.\n* Structured into **small cells** for easier debugging in a notebook.\n'

In [ ]:
#|export
from __future__ import annotations
import os, asyncio, html
from typing import AsyncIterator, List, Dict

In [ ]:
#|export
# ── FastHTML / HTMX scaffold ───────────────────────────────────────────────
from fasthtml import FastHTML
from fasthtml.common import Div, Form, Input, Button, H1
from starlette.requests import Request
from sse_starlette.sse import EventSourceResponse  # proper SSE helper


In [ ]:
#|export
from agents import Agent, Runner
from agents.mcp import MCPServerSse

In [ ]:
#|export
# ── FastHTML app ───────────────────────────────────────────────────────────
app = FastHTML(
    hdrs=(
        '<script src="https://cdn.tailwindcss.com"></script>',
        '<script src="https://cdn.jsdelivr.net/npm/daisyui@4/dist/full.min.js"></script>',
        '<script src="https://unpkg.com/htmx-ext-sse@2.2.3/dist/sse.js"></script>',
    ),
    live=True,
)

In [ ]:
#|export
# ── in‑memory chat log & lock ──────────────────────────────────────────────
MSG: List[Dict[str, str]] = []
MSG_LOCK = asyncio.Lock()

In [ ]:
#|export
# ── UI helper: chat bubble ─────────────────────────────────────────────────

def _chat_bubble(idx: int, **hx):
    """Return one chat bubble (`Div`)."""
    who, txt = MSG[idx]["role"], MSG[idx]["content"] or "…"
    side   = "chat-end"  if who == "user"      else "chat-start"
    bubble = "bg-sky-700 text-white" if who == "assistant" else "bg-gray-200"
    return Div(
        Div(who, cls="chat-header text-xs text-gray-500"),
        Div(txt if who == "user" else html.unescape(txt),
            cls=f"chat-bubble {bubble}", **hx),
        cls=f"chat {side}", id=f"m{idx}",
    )

In [ ]:
#|export
# ── UI helper: text input ──────────────────────────────────────────────────

def _chat_input():
    """Return the text input element (FastHTML ≥ 0.3.1 supports `hx_swap_oob`)."""
    return Input(
        id="msgin", name="msg", type="text", autocomplete="off",
        placeholder="Type your question…",
        cls="input input-bordered w-full",
        hx_swap_oob="true",  # FastHTML fork already fixes the colon bug
        onkeyup="event.key==='Enter' && this.form.requestSubmit()",
    )

In [ ]:
#|export
# ── Route: home page ───────────────────────────────────────────────────────

@app.route("/")
async def home():
    """Root page (GET) – delivers the chat UI shell."""
    return Div(
        H1("Arena Buddy", cls="text-3xl font-bold mb-4"),
        Div(id="chatlog",
            cls="space-y-3 mb-4 h-[70vh] overflow-y-auto bg-base-200 p-4 rounded-box"),
        Form(
            Div(_chat_input(),
                Button("Send ✈", cls="btn btn-primary ml-2"),
                cls="flex"),
            hx_post="/send", hx_target="#chatlog", hx_swap="beforeend",
        ),
        cls="max-w-2xl mx-auto p-6",
    )

In [ ]:
# ── LLM helper ─────────────────────────────────────────────────────────────

async def _assistant_html(prompt: str) -> str:
    """Run the LLM agent and return **raw HTML** (no escaping)."""
    async with MCPServerSse(name="ui", params={"url": MCP_URL}) as srv:
        agent = Agent(
            "assistant",
            instructions=(
                "You are Arena Buddy: convince the human to reach Metro Areena Espoo "
                "with the lowest‑emission mode possible (walk 🚶, bike 🚴, public "
                "transport 🚇, car 🚗 as last resort). Retrieve live routes, parking "
                "status and weather via MCP tools and reply in short friendly "
                "paragraphs."
            ),
            mcp_servers=[srv],
            tool_choice="required",
        )
        result = await Runner.run(starting_agent=agent, input=prompt)
        return result.final_output

In [ ]:
#|export
# ── Endpoint: /send ────────────────────────────────────────────────────────

@app.post("/send")
async def _send(request: Request):
    """Handle user message; return immediate DOM diff (user bubble + empty assistant)."""
    form = await request.form()
    prompt = str(form.get("msg", "")).strip()
    if not prompt:
        return ""  # HTMX: empty response → do nothing

    async with MSG_LOCK:
        MSG.append({"role": "user", "content": html.escape(prompt)})
        MSG.append({"role": "assistant", "content": ""})
        user_idx, asst_idx = len(MSG) - 2, len(MSG) - 1

    user_html = _chat_bubble(user_idx).__html__()
    asst_html = _chat_bubble(
        asst_idx,
        hx_ext="sse",
        sse_connect=f"/stream/{asst_idx}",
        sse_swap="message",
        sse_close="close",
        hx_swap="innerHTML",
    ).__html__()

    return user_html + asst_html + _chat_input().__html__()

In [ ]:
#|export
# ── Endpoint: /stream/{idx} ────────────────────────────────────────────────

@app.get("/stream/{idx}")
async def _stream(idx: int):
    """SSE endpoint that streams the assistant reply and then closes."""
    async def gen() -> AsyncIterator[str]:
        async with MSG_LOCK:
            prompt_html = MSG[idx - 1]["content"]

        reply_html = await _assistant_html(prompt_html)
        cleaned = reply_html.replace("\n", " ")  # pre‑clean to dodge f‑string backslash rule

        async with MSG_LOCK:
            MSG[idx]["content"] = reply_html

        yield f"event: message\ndata: {cleaned}\n\n"
        yield "event: close\ndata:\n\n"

    return EventSourceResponse(gen())

In [ ]:

# %% [markdown]
# **Optional:** run a local dev server.
# ```python
# if __name__ == "__main__":
#     import uvicorn, logging
#     logging.basicConfig(level="INFO")
#     uvicorn.run(app, host="0.0.0.0", port=8000)
# ```